In [ ]:
!pip install tflearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 107 kB 15.5 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=73d927fb0dc8b648c1e0bbeeddfd567f04da79038b25874b15959b6e52dfade9
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn


In [ ]:
import json 


In [ ]:
intents = {"intents": [
        {"tag": "greeting",
         "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day", "Hey", "What's up?", "Wassup"],
         "responses": ["Hello, is everything all right?", "Good to see you again", "Hi there, how was your day?"],
        },
        {"tag": "goodbye",
         "patterns": ["Bye", "See you later", "Goodbye"],
         "responses": ["See you later, thanks for visiting", "Have a nice day", "Bye! Come back again soon."]
        },
        {"tag": "thanks",
         "patterns": ["Thanks", "Thank you", "That's helpful"],
         "responses": ["Happy to help!", "Any time!", "My pleasure", "No problem"]
        },
        {"tag": "Agree",
         "patterns": ["Yeah, I completely agree", "I know right?"],
         "responses": ["Yeah, I completely agree", "I know right?", "I also like doing that"]
        },
        {"tag": "Name",
         "patterns": ["What is your name?", "what are you called?", "Name?" ],
         "responses": ["My name is MindME", "It is MindME", "Call me MindME"]
        },
        {"tag": "Maker",
         "patterns": ["Who made you?", "Codable?", "Who created you?"],
         "responses": ["Codable ML team created me", "Max Min, Yunsu Han, Hangoo Kang, Sangyun Lee from UIUC created me"]
        }
   ]
}

In [ ]:
with open("intent.json", "w") as outfile: 
  json.dump(intents, outfile)

In [ ]:
MBTI = {"MBTI questions": [
        {"tag": "E vs I",
         "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day", "Hey", ""],
         "responses": ["Hello, thanks for visiting", "Good to see you again", "Hi there, how can I help?"],
         "context_set": ""
        },
        {"tag": "N vs S",
         "patterns": ["Bye", "See you later", "Goodbye"],
         "responses": ["See you later, thanks for visiting", "Have a nice day", "Bye! Come back again soon."]
        },
        {"tag": "F vs T",
         "patterns": ["Thanks", "Thank you", "That's helpful"],
         "responses": ["Happy to help!", "Any time!", "My pleasure"]
        },
        {"tag": "J vs P",
         "patterns": ["What hours are you open?", "What are your hours?", "When are you open?" ],
         "responses": ["We're open every day 9am-9pm", "Our hours are 9am-9pm every day"]
        }
   ]
}

In [ ]:
with open("MBTI.json", "w") as outfile: 
  json.dump(MBTI, outfile)

#Tokenisation and preprocessing

In [ ]:
import pickle
import json
import random
import tensorflow
import tflearn
import numpy
import nltk
import tempfile
import os

from nltk.stem.lancaster import LancasterStemmer
nltk.download('punkt')

with open('intent.json') as file:
    intent_data = json.load(file)

#with open('MBTI.json') as file :
#    mbti_Data = json.load(file)

stemmer = LancasterStemmer()

try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)

except:
    words = []
    labels = []
    #classes = []
    docs_x = []
    docs_y = []
    #ignore_words = ['?']

    for intent in intent_data['intents']:
        for pattern in intent['patterns']:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent['tag'] not in labels:
            labels.append(intent['tag'])

    # stem and lower each word and remove duplicates
    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    print (len(docs_y), "intents")
    print (len(labels), "labels", labels)
    print (len(words), "unique stemmed words", words)


    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    # training set, bag of words for each sentence
    for x, doc in enumerate(docs_x):
        bag = []
        wrds = [stemmer.stem(w.lower()) for w in doc]

        # create bag of words array.
        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)

    training = numpy.array(training)
    output = numpy.array(output)


Instructions for updating:
non-resource variables are not supported in the long term
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


22 intents
6 labels ['Agree', 'Maker', 'Name', 'goodbye', 'greeting', 'thanks']
36 unique stemmed words ["'s", ',', 'agr', 'anyon', 'ar', 'bye', 'cal', 'cod', 'complet', 'cre', 'day', 'good', 'goodby', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'is', 'know', 'lat', 'mad', 'nam', 'right', 'see', 'thank', 'that', 'ther', 'up', 'wassup', 'what', 'who', 'yeah', 'yo', 'you']


In [ ]:
with open("data.pickle", "wb") as f:
  pickle.dump((words, labels, training, output), f)

# Build and Train model

In [ ]:
tensorflow.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net, checkpoint_path='model.tfl.ckpt')

model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Streaming output truncated to the last 5000 lines.
| Adam | epoch: 286 | loss: 0.86468 - acc: 0.7571 -- iter: 22/22
--
Training Step: 859  | total loss: 0.84214 | time: 0.005s
| Adam | epoch: 287 | loss: 0.84214 - acc: 0.7564 -- iter: 08/22
Training Step: 860  | total loss: 0.86601 | time: 0.010s
| Adam | epoch: 287 | loss: 0.86601 - acc: 0.7308 -- iter: 16/22
Training Step: 861  | total loss: 0.88296 | time: 0.017s
| Adam | epoch: 287 | loss: 0.88296 - acc: 0.7244 -- iter: 22/22
--
Training Step: 862  | total loss: 0.86448 | time: 0.005s
| Adam | epoch: 288 | loss: 0.86448 - acc: 0.7144 -- iter: 08/22
Training Step: 863  | total loss: 0.82759 | time: 0.010s
| Adam | epoch: 288 | loss: 0.82759 - acc: 0.6854 -- iter: 16/22
Training Step: 864  | total loss: 1.05409 | time: 0.017s
| Adam | epoch: 288 | loss: 1.05409 - acc: 0.6854 -- iter: 22/22
--
Training Step: 865  | total loss: 1.05018 | time: 0.012s
| Adam | epoch: 289 | loss: 1.05018 - acc: 0.6835 -- iter: 08/22
Training Step: 866  |

In [ ]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)

    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    
    return sentence_words

def bag_of_words(s, words, show_details = False):

    sentence_words = clean_up_sentence(s)

    bag = [0]*len(words)

    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return numpy.array(bag)  

In [ ]:
context = {}

ERROR_THRESHOLD = 0.25

In [ ]:
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bag_of_words(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((labels[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, show_details=False):
    userID = input('Enter your ID: ')
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intent_data['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0)

def chat(show_details=False):
    userID = input('Enter your ID: ')
    print("Start talking with the bot (type quit to stop)!")
    # responses = ''
    while True:
        inp = input("You: ")
        # responses = responses + inp + " "
        if inp.lower() == "quit":
            break

        results = classify(inp)
        if results:
        # loop as long as there are matches to process
          while results:
              for i in intent_data['intents']:
                  # find a tag matching the first result
                  if i['tag'] == results[0][0]:
                      # set context for this intent if necessary
                      if 'context_set' in i:
                          if show_details: print ('context:', i['context_set'])
                          context[userID] = i['context_set']

                      # check if this intent is contextual and applies to this user's conversation
                      if not 'context_filter' in i or \
                          (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                          if show_details: print ('tag:', i['tag'])
                          # a random response from the intent
                          print(random.choice(i['responses']))

              results.pop(0)
 
#    return responses


In [ ]:
chat()
#print(response)